In [ ]:
import sys
import os

# Add NQCC folder to Python path
sys.path.append(os.path.abspath(os.path.join("..", "..")))

from Milestone1.plotter import plot_energy_levels
from Milestone1.levels import load_ion_levels

data = load_ion_levels("C:\Users\iks14535\NQCC\Week3\plotter.py")

# plot without any Zeeman ticks
plot_energy_levels(data, B=0.1)